### ** Imports **

In [1]:
from bsa_core.io import load
from joblib import load as load_prob
from scipy.stats import wilcoxon
import numpy as np
import pylab as pl
pl.gray()

### ** Auxiliary Function **

_ Inputs_

* __pred -__ the  binary segmentation;
* __gt -__ the ground truth; 
* __mask -__ the binary mask; 
* __ prob -__ the probability map (if available).

_ Outputs _

* __sen -__ Sensibility;
* __spec -__ Specificity; 
* __acc -__ Accuracy; 
* __ auc -__ Area Under the ROC curve.

In [2]:
from sklearn.metrics import confusion_matrix, roc_auc_score

def perform_metrics (pred, gt, mask, prob = []):
    
    # Suppressing background regions.
    pred = pred[mask > 0]
    gt = gt[mask > 0]

    # Building confusion matrix.
    # # Cij is the number of observations known to be in group i but predicted to be in group j.
    c_matrix = confusion_matrix(gt, pred)
    
    # Calculating ratios.
    tn = c_matrix[0,0]
    tp = c_matrix[1,1]
    fn = c_matrix[1,0]
    fp = c_matrix[0,1]
    
    # Finding the metrics.
    sen  = tp/(tp + fn)
    spec = tn/(tn + fp)
    acc  = (tp + tn)/(tp + tn + fp + fn)
    
    if len (prob) != 0:
        
        prob = prob[mask > 0]
        auc  = roc_auc_score (gt, prob)
        return sen, spec, acc, auc
    
    return sen, spec, acc

### ** Loading **

In [3]:
test_cases = [1, 2, 3, 4, 5, 44, 77, 81, 82, 139, 162, 163, 235, 236, 239, 240, 255, 291, 319, 324]
r_width, r_height = 700, 605

In [4]:
gts = np.zeros((len(test_cases), r_height, r_width))
masks = np.zeros ((len(test_cases), r_height, r_width))

In [5]:
# Hoover (2000)
segs_hoover = np.zeros((len(test_cases), r_height, r_width))

In [6]:
# Nguyen (2013)
segs_nguyen = np.zeros((len(test_cases), r_height, r_width))
probs_nguyen = np.zeros((len(test_cases), r_height, r_width))

In [7]:
# Azzopardi (2015)
segs_azzopardi = np.zeros((len(test_cases), r_height, r_width))

In [8]:
# Roychowdhury (2015b)
segs_roychowdhury_b = np.zeros((len(test_cases), r_height, r_width))

In [9]:
# Roychowdhury (2015a)
segs_roychowdhury_a = np.zeros((len(test_cases), r_height, r_width))

In [10]:
# Liskowsi (2016)
segs_liskowski = np.zeros((len(test_cases), r_height, r_width))
probs_liskowski = np.zeros((len(test_cases), r_height, r_width))

In [11]:
# Fu (2016)
segs_fu = np.zeros((len(test_cases), r_height, r_width))
probs_fu = np.zeros((len(test_cases), r_height, r_width))

In [12]:
# Orlando (2017)
segs_orlando = np.zeros((len(test_cases), r_height, r_width))

In [13]:
# This work (2018)
segs = np.zeros((len(test_cases), r_height, r_width))
probs = np.zeros((len(test_cases), r_height, r_width))

In order for you to run the cell below, please download the resources from the links below and change the paths accordingly.

* [Hoover (2000)](http://cecas.clemson.edu/~ahoover/stare/probing/index.html)
* [Nguyen (2013)](https://people.eng.unimelb.edu.au/thivun/projects/retinal_segmentation/)
* [Azzopardi (2015)](http://www.cs.rug.nl/~nick/results/)
* [Roychowdhury (2015b)](https://sites.google.com/a/uw.edu/src/useful-links)
* [Roychowdhury (2015a)](https://sites.google.com/a/uw.edu/src/useful-links)
* [Liskowsi (2016)](http://www.cs.put.poznan.pl/kkrawiec/?n=Site.2015IEEETMI)
* [Fu (2016)](http://hzfu.github.io/proj_deepvessel.htmlSH)
* [Orlando (2017)](http://homes.esat.kuleuven.be/~mblaschk/projects/retina/)

In [14]:
gts_path = '../resources/gts/STARE/'
masks_path = '../resources/masks/STARE/'
hoover_path = '../resources/hoover_2000/STARE/'
nguyen_path = '../resources/nguyen_2013/STARE/'
azzopardi_path = '../resources/azzopardi_2015/STARE/'
roychowdhury_b_path = '../resources/roychowdhury_2015b/STARE/'
roychowdhury_a_path = '../resources/roychowdhury_2015a/STARE/'
liskowski_path = '../resources/liskowski_2016/STARE/'
fu_path = '../resources/fu_2016/STARE/'
orlando_path = '../resources/orlando_2017/STARE/'

for i, case in enumerate(test_cases):
    
    # Ground-truth and masks
    gt = load(gts_path + 'im%04d.ah.ppm.gz' %case, normalize = True)
    gts[i] = gt
    mask = load(masks_path + 'mask_%04d.png' %case)
    masks[i] = mask
    
    # Hoover (2000)
    seg_hoover = load(hoover_path + 'im%04d-vessels4.ppm.gz' %case)
    segs_hoover[i] = seg_hoover
    
    # Nguyen (2013)
    seg_nguyen = load(nguyen_path + 'stare_im%04d_multi_scales_unsupervised.png' %case, normalize = True)
    segs_nguyen[i] = seg_nguyen
    prob_nguyen = load(nguyen_path + 'stare_im%04d_multi_scales_unsupervised_soft.png' %case, normalize = True)
    probs_nguyen[i] = prob_nguyen
    
    # Azzopardi (2015)
    seg_azzopardi = load(azzopardi_path + 'im%04d.gif' %case, normalize = True)[:,:,0]
    segs_azzopardi[i] = seg_azzopardi
    
    # Roychowdhury (2015b)
    seg_roychowdhury_b = load(roychowdhury_b_path + 'im%04d.tif' %case, normalize = True)
    segs_roychowdhury_b[i] = seg_roychowdhury_b
    
    # Roychowdhury (2015a)
    seg_roychowdhury_a = load(roychowdhury_a_path + 'im%04d.gif' %case, normalize = True)
    
    if seg_roychowdhury_a.shape == (605,700,3):
        seg_roychowdhury_a = seg_roychowdhury_a[:,:,0]
    segs_roychowdhury_a[i] = seg_roychowdhury_a
    
    # Liskowski (2016)
    prob_liskowski = 1 - load(liskowski_path + 'im%04d.png' %case, normalize = True)[27:632, 27:727]
    probs_liskowski[i] = prob_liskowski
    seg_liskowski = np.where(prob_liskowski < 0.5, 0, 1)
    segs_liskowski[i] = seg_liskowski
    
    # Fu (2016)
    seg_fu = load(fu_path + 'im%04d.ppm_seg_result.png' %case, normalize = True)
    segs_fu[i] = seg_fu
    prob_fu = load(fu_path + 'im%04d.ppm_CRF_result.png' %case, normalize = True)
    probs_fu[i] = prob_fu
    
    # Orlando (2017)
    seg_orlando = load(orlando_path + '%02d_test_fccrf.png' %(i+1), normalize = True)
    segs_orlando[i] = seg_orlando
    
    # This work (2018)
    seg = load('../resources/binary_segmentations/STARE/seg_%04d.png' %case, normalize = True)
    prob = load_prob('../resources/probability_maps/STARE/prob_%04d.npy' %case)[1]
    segs[i] = seg
    probs[i] = prob

### **Calculating metrics**

In [15]:
def get_metrics (segs, gts, masks, probs = []):

    if len(probs) != 0:

        sen_list, spec_list, acc_list, auc_list = [], [], [], []

        for i in range (len(gts)):
            sen, spec, acc, auc = perform_metrics (segs[i], gts[i], masks[i], probs[i])
            sen_list.append(sen)
            spec_list.append(spec)
            acc_list.append(acc)
            auc_list.append(auc)
            
        return sen_list, spec_list, acc_list, auc_list
            
    else:
    
        sen_list, spec_list, acc_list = [], [], []

        for i in range (len(gts)):
            sen, spec, acc = perform_metrics (segs[i], gts[i], masks[i])
            sen_list.append(sen)
            spec_list.append(spec)
            acc_list.append(acc)
            
        return sen_list, spec_list, acc_list


In [16]:
sen_list_hoover, spec_list_hoover, acc_list_hoover = get_metrics (segs_hoover, gts, masks)

In [17]:
sen_list_nguyen, spec_list_nguyen, acc_list_nguyen, auc_list_nguyen = get_metrics (segs_nguyen, gts, masks, probs_nguyen)

In [18]:
sen_list_azzopardi, spec_list_azzopardi, acc_list_azzopardi = get_metrics (segs_azzopardi, gts, masks)

In [19]:
for i in range(len(gts)): 
    segs_roychowdhury_b[i] = np.where(segs_roychowdhury_b[i] >= 0.5, 1, 0)

In [20]:
sen_list_roychowdhury_b, spec_list_roychowdhury_b, acc_list_roychowdhury_b = get_metrics (segs_roychowdhury_b, gts, masks)

In [21]:
for i in range(len(gts)): 
    segs_roychowdhury_a[i] = np.where(segs_roychowdhury_a[i] >= 0.5, 1, 0)

In [22]:
sen_list_roychowdhury_a, spec_list_roychowdhury_a, acc_list_roychowdhury_a = get_metrics (segs_roychowdhury_a, gts, masks)

In [23]:
sen_list_liskowski, spec_list_liskowski, acc_list_liskowski, auc_list_liskowski = get_metrics (segs_liskowski, gts, masks, probs_liskowski)

In [24]:
sen_list_fu, spec_list_fu, acc_list_fu, auc_list_fu = get_metrics (segs_fu, gts, masks, probs_fu)

In [25]:
sen_list_orlando, spec_list_orlando, acc_list_orlando = get_metrics (segs_orlando, gts, masks)

In [26]:
sen_list, spec_list, acc_list, auc_list = get_metrics (segs, gts, masks, probs)

### **Making Statistical Comparison**

In [27]:
# Hoover (2000)
_, pvalue = wilcoxon(sen_list_hoover, sen_list)
pvalue

8.8574576878635474e-05

In [28]:
_, pvalue = wilcoxon(spec_list_hoover, spec_list)
pvalue

8.8574576878635474e-05

In [29]:
_, pvalue = wilcoxon(acc_list_hoover, acc_list)
pvalue

0.0019442974027892919

In [30]:
# Nguyen (2013)
_, pvalue = wilcoxon(acc_list_nguyen, acc_list)
pvalue

8.8574576878635474e-05

In [31]:
# Azzopardi (2015)
_, pvalue = wilcoxon(sen_list_azzopardi, sen_list)
pvalue

0.0015072868413933233

In [32]:
_, pvalue = wilcoxon(spec_list_azzopardi, spec_list)
pvalue

0.00012042223841686183

In [33]:
_, pvalue = wilcoxon(acc_list_azzopardi, acc_list)
pvalue

8.8574576878635474e-05

In [34]:
auc_list_azzopardi = [0.9285, 0.9416, 0.9513, 0.9295, 0.9503,
                      0.9684, 0.9643, 0.9726, 0.9680, 0.9667,
                      0.9746, 0.9802, 0.9586, 0.9606, 0.9598,
                      0.9275, 0.9699, 0.9668, 0.9483, 0.9376]

In [35]:
_, pvalue = wilcoxon(auc_list_azzopardi, auc_list)
pvalue

8.8574576878635474e-05

In [36]:
# Roychowdhury (2015b)
_, pvalue = wilcoxon(sen_list_roychowdhury_b, sen_list)
pvalue

8.8574576878635474e-05

In [37]:
_, pvalue = wilcoxon(spec_list_roychowdhury_b, spec_list)
pvalue

0.00077959325753140308

In [38]:
_, pvalue = wilcoxon(acc_list_roychowdhury_b, acc_list)
pvalue

8.8574576878635474e-05

In [39]:
# Roychowdhury (2015a)
_, pvalue = wilcoxon(sen_list_roychowdhury_a, sen_list)
pvalue

0.0001401336206158955

In [40]:
_, pvalue = wilcoxon(spec_list_roychowdhury_a, spec_list)
pvalue

8.8574576878635474e-05

In [41]:
_, pvalue = wilcoxon(acc_list_roychowdhury_a, acc_list)
pvalue

8.8574576878635474e-05

In [42]:
# Liskowski (2016)
_, pvalue = wilcoxon(sen_list_liskowski, sen_list)
pvalue

0.2958775226696384

In [43]:
_, pvalue = wilcoxon(spec_list_liskowski, spec_list)
pvalue

0.5015913016269502

In [44]:
_, pvalue = wilcoxon(acc_list_liskowski, acc_list)
pvalue

0.65415894441714495

In [45]:
_, pvalue = wilcoxon(auc_list_liskowski, auc_list)
pvalue

0.022768743718921809

In [46]:
# Fu (2016)
_, pvalue = wilcoxon(sen_list_fu, sen_list)
pvalue

0.0013245375247995077

In [47]:
_, pvalue = wilcoxon(acc_list_fu, acc_list)
pvalue

8.8574576878635474e-05

In [48]:
# Orlando (2017)
_, pvalue = wilcoxon(sen_list_orlando, sen_list)
pvalue

0.0013245375247995077

In [49]:
_, pvalue = wilcoxon(spec_list_orlando, spec_list)
pvalue

0.00029316154775863669

In [50]:
_, pvalue = wilcoxon(acc_list_orlando, acc_list)
pvalue

8.8574576878635474e-05